In [1]:
import numpy as np
import pandas as pd
import os
import re
import itertools
import collections
import tqdm

In [2]:
arquivos95 = list(os.walk('Bases/folha95/'))[0][2]
arquivos94 = list(os.walk('Bases/folha94/'))[0][2]

In [3]:
corpus = ''
for a in arquivos95:
        with open('Bases/folha95/' + a,'r', encoding='iso-8859-1') as f:
                corpus = corpus + f.read()
for a in arquivos94:
        with open('Bases/folha94/' + a,'r', encoding='iso-8859-1') as f:
                corpus = corpus + f.read()

In [4]:
def tokenize (txt):
    p = re.finditer ('(?:<TEXT>).*?(?:</TEXT>)',txt,flags=re.DOTALL|re.MULTILINE)
    for a in p:
        for x in re.finditer('[a-zA-ZâêîôûáéíóúàèìòùãẽĩõũçÁÉÍÓÚÀÈÌÒÙÃẼĨÕŨÂÊÎÔÛÇ]+|<TEXT>|</TEXT>|\.',a.group(),flags=re.DOTALL|re.MULTILINE):
            t = x.group()
            if t=='<TEXT>':
                yield '<s>'
                yield '<s>'
                yield '<s>'
            elif t == '</TEXT>':
                yield '</s>'
                yield '</s>'
                yield '</s>' 
            elif t == '.':
                yield '</s>'
                yield '</s>'
                yield '</s>'  
                yield '<s>'
                yield '<s>'
                yield '<s>'
            else:
                yield t.lower()

In [5]:
# Total de Palavras
totalpalavras=0
for d in tokenize(corpus):
        totalpalavras+=1

In [6]:
print(totalpalavras)

47687662


In [7]:
# Total de Palavras Distintas
inverseDict={}
total=0
for p in tokenize(corpus):
    if p not in inverseDict:
        inverseDict[p]=total
        total+=1

In [8]:
len(inverseDict)

258504

In [9]:
def slidingWindow(winsize):
    def itt(tokenizedText):
        iters=itertools.tee(tokenizedText,winsize)
        for x in range(winsize):
            for i in range(x):
                next(iters[x])
        while True:
            try:
                yield tuple([next(it) for it in iters])
            except StopIteration:
                return
    return itt

In [10]:
f4 = slidingWindow(4)
quatro = f4(tokenize(corpus))

In [11]:
next(quatro)

('<s>', '<s>', '<s>', 'datafolha')

In [12]:
f4 = slidingWindow(4)
f3 = slidingWindow(3)
f2 = slidingWindow(2)
quatro = f4(tokenize(corpus))
tres = f3(tokenize(corpus))
#tres2 = f3(tokenize(corpus2))
dois = f2(tokenize(corpus))
um = tokenize(corpus)

In [13]:
total = sum(1 for _ in tokenize(corpus))

In [14]:
contador={}

In [15]:
for a,b,c,d in tqdm.tqdm(zip(um,dois,tres,quatro),total=total):
    if a not in contador:
        contador[a]={None: 1}
    else:
        contador[a][None]+=1
        
    if b[1] not in contador[b[0]]:
        contador[b[0]][b[1]]={None: 1}
    else:
        contador[b[0]][b[1]][None]+=1
    
    if c[2] not in contador[c[0]][c[1]]:
        contador[c[0]][c[1]][c[2]] = {None: 1}
    else:
        contador[c[0]][c[1]][c[2]][None] += 1
   
    if d[3] not in contador[d[0]][d[1]][d[2]]:
        contador[d[0]][d[1]][d[2]][d[3]] = 1
    else:
        contador[d[0]][d[1]][d[2]][d[3]] += 1

100%|█████████████████████████████████████████████████████████████████▉| 47687659/47687662 [07:20<00:00, 108239.48it/s]


In [16]:
#contador['<s>']['<s>']['<s>']

In [42]:
def analisaFrase(x):
    xx = tokenize(x)
    tres = f3(tokenize(x))
    dois = f2(tokenize(x))
    next(dois)
    um = tokenize(x)
    next(um)
    next(um)
    prod=0
    # Unigrama
    w1,w2,w3=1,0,0
    # Bigrama
    #w1,w2,w3=0,1,0
    # Trigrama
    #w1,w2,w3=0,0,1
    # Interpolação
    #w1,w2,w3=0.1,0.2,0.7
    i=0
    for w in zip(um, dois,tres):
        p3=0
        p2=0
        
        if contador[w[2][0]][w[2][1]][None] > 0:
            p3 = contador[w[2][0]][w[2][1]][w[2][2]][None]/contador[w[2][0]][w[2][1]][None]
            
        if contador[w[1][0]][None] > 0:
            p2 = contador[w[1][0]][w[1][1]][None]/contador[w[1][0]][None]
            
        p1 = contador[w[0]][None]/total
        
        prod = prod + np.log(w1*p1 + w2*p2 + w3*p3)
        i+=1
    return 1/((np.exp(prod))**(1/i))

In [43]:
#Perplexidade Unigrama
analisaFrase('<TEXT>Mais de uma vez, como candidato, Fernando Henrique Cardoso</TEXT>')

193.10247355629545

In [40]:
def analisaFrase(x):
    xx = tokenize(x)
    tres = f3(tokenize(x))
    dois = f2(tokenize(x))
    next(dois)
    um = tokenize(x)
    next(um)
    next(um)
    prod=0
    # Unigrama
    #w1,w2,w3=1,0,0
    # Bigrama
    w1,w2,w3=0,1,0
    # Trigrama
    #w1,w2,w3=0,0,1
    # Interpolação
    #w1,w2,w3=0.1,0.2,0.7
    i=0
    for w in zip(um, dois,tres):
        p3=0
        p2=0
        
        if contador[w[2][0]][w[2][1]][None] > 0:
            p3 = contador[w[2][0]][w[2][1]][w[2][2]][None]/contador[w[2][0]][w[2][1]][None]
            
        if contador[w[1][0]][None] > 0:
            p2 = contador[w[1][0]][w[1][1]][None]/contador[w[1][0]][None]
            
        p1 = contador[w[0]][None]/total
        
        prod = prod + np.log(w1*p1 + w2*p2 + w3*p3)
        i+=1
    return 1/((np.exp(prod))**(1/i))

In [41]:
#Perplexidade Bigrama
analisaFrase('<TEXT>Mais de uma vez, como candidato, Fernando Henrique Cardoso</TEXT>')

18.71773826661556

In [38]:
def analisaFrase(x):
    xx = tokenize(x)
    tres = f3(tokenize(x))
    dois = f2(tokenize(x))
    next(dois)
    um = tokenize(x)
    next(um)
    next(um)
    prod=0
    # Unigrama
    #w1,w2,w3=1,0,0
    # Bigrama
    #w1,w2,w3=0,1,0
    # Trigrama
    w1,w2,w3=0,0,1
    # Interpolação
    #w1,w2,w3=0.1,0.2,0.7
    i=0
    for w in zip(um, dois,tres):
        p3=0
        p2=0
        
        if contador[w[2][0]][w[2][1]][None] > 0:
            p3 = contador[w[2][0]][w[2][1]][w[2][2]][None]/contador[w[2][0]][w[2][1]][None]
            
        if contador[w[1][0]][None] > 0:
            p2 = contador[w[1][0]][w[1][1]][None]/contador[w[1][0]][None]
            
        p1 = contador[w[0]][None]/total
        
        prod = prod + np.log(w1*p1 + w2*p2 + w3*p3)
        i+=1
    return 1/((np.exp(prod))**(1/i))

In [39]:
#Perplexidade Trigrama
analisaFrase('<TEXT>Mais de uma vez, como candidato, Fernando Henrique Cardoso</TEXT>')

13.990084651039636

In [62]:
def analisaFrase(x):
    xx = tokenize(x)
    tres = f3(tokenize(x))
    dois = f2(tokenize(x))
    next(dois)
    um = tokenize(x)
    next(um)
    next(um)
    prod=0
    w1,w2,w3=0.1,0.2,0.7
    i=0
    for w in zip(um,dois,tres):
        p3=0
        p2=0
        
        if contador[w[2][0]][w[2][1]][None] > 0:
            p3 = contador[w[2][0]][w[2][1]][w[2][2]][None]/contador[w[2][0]][w[2][1]][None]
            
        if contador[w[1][0]][None] > 0:
            p2 = contador[w[1][0]][w[1][1]][None]/contador[w[1][0]][None]
            
        p1 = contador[w[0]][None]/total
        
        prod = prod + np.log(w1*p1 + w2*p2 + w3*p3)
        i+=1
    return 1/((np.exp(prod))**(1/i))

In [84]:
#Perplexidade Interpolação
analisaFrase('<TEXT>das</TEXT>')

11.864636091986256